In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.cluster.hierarchy import dendrogram, cophenet
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import pdist

In [2]:
# Load the dataset
df = pd.read_csv('churn_clean.csv', index_col=0)
#Check Data Types
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer_id           10000 non-null  object 
 1   Interaction           10000 non-null  object 
 2   UID                   10000 non-null  object 
 3   City                  10000 non-null  object 
 4   State                 10000 non-null  object 
 5   County                10000 non-null  object 
 6   Zip                   10000 non-null  int64  
 7   Lat                   10000 non-null  float64
 8   Lng                   10000 non-null  float64
 9   Population            10000 non-null  int64  
 10  Area                  10000 non-null  object 
 11  TimeZone              10000 non-null  object 
 12  Job                   10000 non-null  object 
 13  Children              10000 non-null  int64  
 14  Age                   10000 non-null  int64  
 15  Income                10

In [3]:
df.head(5)

,Customer_id,Interaction,UID,City,State,County,Zip,Lat,Lng,Population,...,MonthlyCharge,Bandwidth_GB_Year,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8
CaseOrder,,,,,,,,,,,,,,,,,,,,,
1,K409198,aa90260b-4141-4a24-8e36-b04ce1f4f77b,e885b299883d4f9fb18e39c75155d990,Point Baker,AK,Prince of Wales-Hyder,99927,56.25100,-133.37571,38,...,172.455519,904.536110,5,5,5,3,4,4,3,4
2,S120509,fb76459f-c047-4a9d-8af9-e0f7d4ac2524,f2de8bef964785f41a2959829830fb8a,West Branch,MI,Ogemaw,48661,44.32893,-84.24080,10446,...,242.632554,800.982766,3,4,3,3,4,3,4,4
3,K191035,344d114c-3736-4be5-98f7-c72c281e2d35,f1784cfa9f6d92ae816197eb175d3c71,Yamhill,OR,Yamhill,97148,45.35589,-123.24657,3735,...,159.947583,2054.706961,4,4,2,4,4,3,3,3
4,D90850,abfa2b40-2d43-4994-b15a-989b8c79e311,dc8a365077241bb5cd5ccd305136b05e,Del Mar,CA,San Diego,92014,32.96687,-117.24798,13863,...,119.956840,2164.579412,4,4,4,2,5,4,3,3
5,K662701,68a861fd-0d20-4e51-a587-8a90407ee574,aabb64a116e83fdc4befc1fbab1663f9,Needville,TX,Fort Bend,77461,29.38012,-95.80673,11352,...,149.948316,271.493436,4,4,4,3,4,4,4,5


In [4]:
# Handle missing values and outliers
print("Initial Missing Values:")
print(df.isnull().sum())

# Identify columns with missing values
columns_with_missing = df.columns[df.isnull().any()]

# Replace missing values or drop rows/columns as needed
# Impute missing values for numerical columns (replace with mean)
for col in columns_with_missing:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        df[col].fillna(df[col].mean(), inplace=True)

# Impute missing values for categorical columns (replace with mode)
for col in columns_with_missing:
    if df[col].dtype == 'object':
        df[col].fillna(df[col].mode()[0], inplace=True)
# Display information about missing values after imputation
print("\nMissing Values After Imputation:")
print(df.isnull().sum())

Initial Missing Values:
Customer_id                0
Interaction                0
UID                        0
City                       0
State                      0
County                     0
Zip                        0
Lat                        0
Lng                        0
Population                 0
Area                       0
TimeZone                   0
Job                        0
Children                   0
Age                        0
Income                     0
Marital                    0
Gender                     0
Churn                      0
Outage_sec_perweek         0
Email                      0
Contacts                   0
Yearly_equip_failure       0
Techie                     0
Contract                   0
Port_modem                 0
Tablet                     0
InternetService         2129
Phone                      0
Multiple                   0
OnlineSecurity             0
OnlineBackup               0
DeviceProtection           0
TechSupport        

In [5]:
#Categorical Columns to Numeric
categorical_columns = ['Techie','Port_modem','Churn','OnlineSecurity','DeviceProtection', 'TechSupport', 'PaperlessBilling', 'Tablet', 'Phone', 'Multiple', 'OnlineBackup', 'StreamingTV', 'StreamingMovies']
for column in categorical_columns:
    df[column] = df[column].map({'Yes': 1, 'No': 0})

In [6]:
area_mapping = {'Urban': 0,'Suburban': 1,'Rural': 2}
marital_mapping = {'Widowed': 0,'Married': 1,'Separated': 2,'Never Married': 3,'Divorced' :4}
gender_mapping = {'Male':0,'Female': 1,'Prefer not to answer': 2}
contract_mapping = {'One year': 0,'Month-to-month': 1,'Two Year': 2}
InternetService_mapping = {'DSL': 0,'Fiber Optic': 1,'None': 2}
PaymentMethod_mapping = {'Bank Transfer(automatic)': 0,'Credit Card (automatic)': 1,'Mailed Check': 2, 'Electronic Check': 3}



df['Area'] = df['Area'].map(area_mapping)
df['Marital'] = df['Marital'].map(marital_mapping)
df['Gender'] = df['Gender'].map(gender_mapping)
df['Contract'] = df['Contract'].map(contract_mapping)
df['InternetService'] = df['InternetService'].map(InternetService_mapping)
df['PaymentMethod'] = df['PaymentMethod'].map(PaymentMethod_mapping)

In [7]:
# Establish mapping to reverse survey questions (1 < 8)
survey_mapping = {1: 8, 2: 7, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1}

# Loop through survey items, remap columns, and convert to float
survey_items = ["Item1", "Item2", "Item3", "Item4", "Item5", "Item6", "Item7", "Item8"]
for item in survey_items:
    df[item] = df[item].map(survey_mapping).astype('float64')


In [8]:
#Missing Values
df['Gender']= df['Gender'].fillna(df['Gender'].mode()[0])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 1 to 10000
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer_id           10000 non-null  object 
 1   Interaction           10000 non-null  object 
 2   UID                   10000 non-null  object 
 3   City                  10000 non-null  object 
 4   State                 10000 non-null  object 
 5   County                10000 non-null  object 
 6   Zip                   10000 non-null  int64  
 7   Lat                   10000 non-null  float64
 8   Lng                   10000 non-null  float64
 9   Population            10000 non-null  int64  
 10  Area                  10000 non-null  int64  
 11  TimeZone              10000 non-null  object 
 12  Job                   10000 non-null  object 
 13  Children              10000 non-null  int64  
 14  Age                   10000 non-null  int64  
 15  Income                10

In [9]:
for i in range(1, 9):
    mean_score = round(df[f'Item{i}'].mean(), 3)
    std_deviation = round(df[f'Item{i}'].std(), 3)
    print(f"Question {i}: Mean score - {mean_score}, Standard deviation - {std_deviation}")

Question 1: Mean score - 5.509, Standard deviation - 1.038
Question 2: Mean score - 5.495, Standard deviation - 1.035
Question 3: Mean score - 5.513, Standard deviation - 1.028
Question 4: Mean score - 5.502, Standard deviation - 1.026
Question 5: Mean score - 5.507, Standard deviation - 1.025
Question 6: Mean score - 5.503, Standard deviation - 1.034
Question 7: Mean score - 5.49, Standard deviation - 1.029
Question 8: Mean score - 5.504, Standard deviation - 1.029


In [11]:
reduced_df = df[survey_items]

In [12]:
reduced_df.to_csv('clean.csv', index=False)

In [13]:
selected_columns = ['Item1', 'Item2', 'Item3', 'Item4', 'Item5', 'Item6', 'Item7', 'Item8'] 
##selected_columns = ['Age', 'Income', 'MonthlyCharge', 'Bandwidth_GB_Year']
X = df[selected_columns]


In [14]:
# Hierarchical Clustering
linkage_matrix = linkage(X, method='ward', metric='euclidean')
# Calculate cophenetic correlation coefficient
coph_coeff, coph_dists = cophenet(linkage_matrix, pdist(X, metric='euclidean'))
print(f'Cophenetic Correlation Coefficient: {coph_coeff}')


Cophenetic Correlation Coefficient: 0.3816549031685567


In [ ]:
# Create Dendrogram
plt.figure(figsize=(16, 5))
dendrogram(linkage_matrix)
plt.title('Dendrogram')
plt.xlabel('Customer Index')
plt.ylabel('Distance')
plt.show()

In [ ]:
# Assign cluster labels
df['ward_cluster_labels'] = fcluster(linkage_matrix, 3, criterion='maxclust')
print(df['ward_cluster_labels'].value_counts().sort_index())

In [ ]:
# Set the figure size
plt.figure(figsize=[20, 10])

# Create subplots for each survey item
for i, item in enumerate(["Item1", "Item2", "Item3", "Item4", "Item5", "Item6", "Item7", "Item8"], start=1):
    plt.subplot(2, 4, i)
    plt.title(f'Distribution of {item} Scores by Cluster Label')
    sns.countplot(data=df, x=item, hue="ward_cluster_labels")
    plt.legend(["Cluster 1", "Cluster 2", "Cluster 3"])
    plt.xlabel(f"{item} Scores")
    plt.ylabel("Number of Customers")

# Adjust layout
plt.tight_layout()

# Calculate means for each survey item by cluster
for item in ["Item1", "Item2", "Item3", "Item4", "Item5", "Item6", "Item7", "Item8"]:
    for cluster_label in range(1, 4):
        cluster_mean = df.loc[df['ward_cluster_labels'] == cluster_label, item].mean()
        print(f"\nFor {item}, respondents from Cluster {cluster_label} scored this at {round(cluster_mean, 3)}, on average.")

In [ ]:
# Calculate means for each survey item by cluster
summary_dict = {'cluster_1': [], 'cluster_2': [], 'cluster_3': []}

for item in ["Item1", "Item2", "Item3", "Item4", "Item5", "Item6", "Item7", "Item8"]:
    for cluster_label in range(1, 4):
        cluster_mean = df.loc[df['ward_cluster_labels'] == cluster_label, item].mean()
        summary_dict[f'cluster_{cluster_label}'].append(round(cluster_mean, 3))

# Create a summary DataFrame
summary_df = pd.DataFrame(data=summary_dict, index=['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8'])

# Print the summary DataFrame
print("A summary of the means for each question:")
print(summary_df)

In [ ]:
plt.figure(figsize = [16,5])
sns.lineplot(data = summary_df, markers=True)
plt.title("Distribution of Average Survey Scores per Question")
plt.xlabel("Survey Question Number")
plt.ylabel("Mean Score");

In [ ]:
# Define and X (feature columns) and y (resulting cluster labels)
X = df[["Item1", "Item2", "Item3", "Item4", "Item5", "Item6", "Item7", "Item8"]]
y = df['ward_cluster_labels']
model_score = silhouette_score(X, y, metric='euclidean')
print(f"The silhouette score of this hierarchical clustering is: {round(model_score, 3)}")